In [10]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update

# Read the airline data into pandas dataframe

spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# REVIEW1: Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                            dcc.Dropdown(id='site-dropdown',
                                              options=[
                                                  {'label': 'All Sites', 'value': 'ALL'},
                                                  {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                  {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                  {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                  {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                              ],
                                            value='ALL',
                                            placeholder="place holder here",
                                            searchable=True
                                            ),                 
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                       100: '100'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value')])

def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig1 = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches By Site')
        return fig1
    
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        filtered_df = filtered_df
        fig1 = px.pie(filtered_df, 
        names='class', 
        title='Total Success Launches For Site {}'.format(entered_site))
        return fig1

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
             [Input(component_id='site-dropdown', component_property='value'), 
             Input(component_id="payload-slider", component_property="value")])

def get_scatter_plot(entered_site, payload_range):
   
    filtered_df_1 = spacex_df[(spacex_df['Payload Mass (kg)']> payload_range[0]) 
                              & (spacex_df['Payload Mass (kg)']<payload_range[1])]
    
    if entered_site == 'ALL':
        fig2 = px.scatter(filtered_df_1, x = 'Payload Mass (kg)',
        y = 'class',color = "Booster Version Category",
        title = 'Correlation Between Payload and Success for All Sites')
        return fig2
    
    else:
        
        filtered_df =  filtered_df_1[filtered_df_1['Launch Site'] == entered_site]
        filtered_df = filtered_df
        fig2 = px.scatter(filtered_df, x = 'Payload Mass (kg)',
        y = 'class',color = "Booster Version Category",
        title = 'Correlation Between Payload and Success for Site {}'.format(entered_site) )
        return fig2

# Run the app
if __name__ == '__main__':
    app.run_server(port=8051)


Dash is running on http://127.0.0.1:8051/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [04/May/2023 20:25:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2023 20:25:45] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2023 20:25:45] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2023 20:25:45] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2023 20:25:45] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2023 20:25:45] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2023 20:25:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2023 20:25:45] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2023 20:25:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2023 20:25:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2023 20:25:48] "POST /_dash-

## A1: Which site has the largest successful launches?
## A1: Site KSC-LC-39A has the largest number of successful launches with 10

## Q2: Which site has the largest successful launch rate?
## A2: Site KSC-LC-39A has the highest launch success rate with 76.9%

## Q3: Which payload range(s) has the highest launch success rate?
## A3: The range 0-4000 kg has the highest launch success rate

## Q4: Which payload range(s) has the lowest launch success rate?
## A4: The range 5000-10000 kg has the lowest launch success rate

## Q5: Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?
## A5: FT Boostyer Version has the highest success rate

In [9]:
spacex_df.groupby('Launch Site')['class'].value_counts()

Launch Site   class
CCAFS LC-40   0        19
              1         7
CCAFS SLC-40  0         4
              1         3
KSC LC-39A    1        10
              0         3
VAFB SLC-4E   0         6
              1         4
Name: class, dtype: int64